<a href="https://colab.research.google.com/github/bird0401/Instance_level_recognition/blob/main/02_resnet_car2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning==0.7.1

     |████████████████████████████████| 6.0 MB 12.1 MB/s 
     |████████████████████████████████| 829 kB 34.2 MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-0.7.1-py3-none-any.whl size=145329 sha256=16769e96e49a9927b0476dc194c0f21f855b5a133a8356c142c94709ea8203e5
  Stored in directory: /root/.cache/pip/wheels/a5/c0/6c/ed64904da20814878f410c520ae61c062c6d7e93bf5c27dcd4
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=784b9e05601b631b8a4e32e59c3f3486e942e4cbfe3a342520ca3706c46b5b9a
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built pytorch-lightning future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [2]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer

import torch,torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.models import resnet18
import torch.utils.data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

from collections import defaultdict
import os
import random
from tqdm import tqdm
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

In [4]:
categories=["car2","uspresident","uspresident_02","tower","tower_02"]
category=categories[0]

In [5]:
# !cp /content/drive/MyDrive/instance_level_recognition/dog_2.tar.gz .
!cp /content/drive/MyDrive/instance_level_recognition/imgs_{category}.tar.gz .

In [8]:
!tar -zxvf imgs_{category}.tar.gz

imgs_car2/Q1125433_069.jpg
imgs_car2/Q926873_110.jpg
imgs_car2/Q214229_089.jpg
imgs_car2/Q370524_034.jpg
imgs_car2/Q370524_244.JPG
imgs_car2/Q917250_017.jpg
imgs_car2/Q222775_030.jpg
imgs_car2/Q826094_072.jpg
imgs_car2/Q22950015_027.jpg
imgs_car2/Q926873_057.jpg
imgs_car2/Q370524_061.jpg
imgs_car2/Q1360944_004.jpg
imgs_car2/Q97183339_016.jpg
imgs_car2/Q97183332_026.jpg
imgs_car2/Q370524_160.JPG
imgs_car2/Q826094_098.jpg
imgs_car2/Q3502379_016.jpg
imgs_car2/Q849464_042.JPG
imgs_car2/Q926873_117.jpg
imgs_car2/Q370524_082.jpg
imgs_car2/Q275747_015.jpg
imgs_car2/Q222775_051.jpg
imgs_car2/Q643635_027.jpg
imgs_car2/Q214229_035.jpg
imgs_car2/Q1511429_030.JPG
imgs_car2/Q370524_252.jpg
imgs_car2/Q826094_086.JPG
imgs_car2/Q1249123_030.jpg
imgs_car2/Q222775_099.JPG
imgs_car2/Q370524_206.jpg
imgs_car2/Q18913449_071.jpg
imgs_car2/Q830375_055.jpg
imgs_car2/Q2003307_074.jpg
imgs_car2/Q222775_014.jpg
imgs_car2/Q1368464_051.jpg
imgs_car2/Q914069_028.jpg
imgs_car2/Q370524_238.jpg
imgs_car2/Q748949_040.j

In [9]:
 !ls imgs_{category}|wc -l

2935


In [10]:
!rm -rf /content/{category}.tar.gz

In [11]:
# # 224×224で正規化した画像に変換

class ImageTranform():
  def __init__(self):
    self.data_transform=transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
  def __call__(self,img):
    return self.data_transform(img)

In [12]:
# transform=ImageTranform()
# img=Image.open("images/Q17504_001.jpg")
# img=transform(img)
# # img=img.numpy().transpose((1,2,0)) # imshow用に次元入れ替え
# img=np.clip(img,0,1)
# plt.imshow(img)
# plt.show()
# print(img.shape)
# print(img.dtype)

In [13]:
# 実行後のファイルはすでに保存してあるのでコメントアウト

# # image_size=32 #試し用
# image_size=224 

# image_shape=(image_size,image_size,3)

# ファイルの入出力用にglobをインポート
import glob

image_dir = "imgs_"+category
paths=glob.glob(image_dir+"/*")

## 準備

# q_files = defaultdict(list)
imgs,qids=[],[]
transform=ImageTranform()

for path in tqdm(paths):
  try:
    # img=np.array(Image.open(path).convert("RGB").resize((image_size,image_size)))
    img=Image.open(path)
    img=transform(img)
    img=img.numpy()
    img=np.clip(img,0,1)
  except Exception as e:
    print(e)
  else:
    ## キーと画像のリストを保存
    filename=path.split("/")[1]
    qid = filename.split("_")[0][1:]
    qids.append(qid)
    imgs.append(img)
    ## キーから画像を取得できるようにする
    # q_files[qid].append(img)

  0%|          | 12/2935 [00:04<11:55,  4.09it/s]

output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]


  3%|▎         | 78/2935 [00:17<10:19,  4.61it/s]

output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]


  4%|▍         | 111/2935 [00:23<11:32,  4.08it/s]

...

 96%|█████████▋| 2830/2935 [09:18<00:26,  3.93it/s]

output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]


 98%|█████████▊| 2885/2935 [09:27<00:04, 11.77it/s]

output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]


100%|██████████| 2935/2935 [09:37<00:00,  5.08it/s]


In [14]:
# #  pickleで保存
with open('qids_'+category+'.binaryfile','wb') as f:
  pickle.dump(qids,f)

# with open('imgs.binaryfile','wb') as f:
#   pickle.dump(imgs,f)

In [15]:
# # 保存したファイルをdriveにコピー
!cp /content/{'qids_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition
# !cp /content/imgs.binaryfile /content/drive/MyDrive/instance_level_recognition

In [16]:
# 保存したファイルをdriveから取ってくる
!cp /content/drive/MyDrive/instance_level_recognition/{'qids_'+category+'.binaryfile'} .
# !cp /content/drive/MyDrive/instance_level_recognition/imgs.binaryfile .

In [17]:
# pickleからロード
with open('/content/qids_'+category+'.binaryfile','rb') as f:
  qids=pickle.load(f)
# with open('/content/imgs.binaryfile','rb') as f:
#   imgs=pickle.load(f)

In [18]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(qids)
y_labels=le.transform(qids)

In [19]:
n_classes=len(set(y_labels.tolist()))

In [20]:
imgs=torch.tensor(imgs,dtype=torch.float32)
y_labels=torch.from_numpy(y_labels.astype(np.uint8)).clone()

# X_train, X_valid, X_test = X_train/255.0, X_valid/255.0, X_test/255.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [23]:
#  pickleで保存
with open('y_labels_'+category+'.binaryfile','wb') as f:
  pickle.dump(y_labels,f)

with open('imgs_'+category+'.binaryfile','wb') as f:
  pickle.dump(imgs,f)

In [24]:
# 保存したファイルをdriveにコピー
!cp /content/y_labels_tensor.binaryfile /content/drive/MyDrive/instance_level_recognition
!cp /content/imgs_tensor.binaryfile /content/drive/MyDrive/instance_level_recognition

cp: cannot stat '/content/y_labels_tensor.binaryfile': No such file or directory
cp: cannot stat '/content/imgs_tensor.binaryfile': No such file or directory


In [25]:
# 保存したファイルをdriveから取ってくる
!cp /content/drive/MyDrive/instance_level_recognition/{'y_labels_'+category+'.binaryfile'} .
!cp /content/drive/MyDrive/instance_level_recognition/'imgs_'+category+'.binaryfile' .

cp: cannot stat '/content/drive/MyDrive/instance_level_recognition/y_labels_car2.binaryfile': No such file or directory
cp: cannot stat '/content/drive/MyDrive/instance_level_recognition/imgs_+category+.binaryfile': No such file or directory


In [27]:
# pickleからロード
with open('y_labels_'+category+'.binaryfile','rb') as f:
  y_labels=pickle.load(f)
with open('imgs_'+category+'.binaryfile','rb') as f:
  imgs=pickle.load(f)

In [28]:
# 統合
dataset=torch.utils.data.TensorDataset(imgs,y_labels)
dataset

In [29]:
n_train = int(len(dataset) * 0.6)
n_val = int((len(dataset) - n_train) * 0.5)
n_test = len(dataset) - n_train - n_val
n_train, n_val, n_test

(1715, 572, 572)

In [30]:
torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])
# len(X_train),len(X_valid),len(X_test)

# Resnet18

In [31]:
class TrainNet(pl.LightningModule):
  def train_dataloader(self):
    return torch.utils.data.DataLoader(train,self.batch_size,shuffle=True)
  def training_step(self,batch,batch_nb):
    x,t=batch
    y=self.forward(x)
    loss=self.lossfun(y,t)
    results={"loss":loss}
    return results

class ValidationNet(pl.LightningModule):
  def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)
  def validation_step(self,batch,batch_nb):
    x,t=batch
    y=self.forward(x)
    loss=self.lossfun(y,t)
    y_label=torch.argmax(y,dim=1)
    acc=torch.sum(t==y_label)/len(t)
    results={"val_loss":loss,"val_acc":acc}
    return results
  def validation_end(self,outputs):
    avg_loss=torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_acc=torch.stack([x["val_acc"] for x in outputs]).mean()
    results={"val_loss":avg_loss,"val_acc":avg_acc}
    return results

class TestNet(pl.LightningModule):
    def test_dataloader(self):
        return torch.utils.data.DataLoader(test, self.batch_size)

    def test_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'test_loss': loss, 'test_acc': acc}
        return results

    def test_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        results = {'test_loss': avg_loss, 'test_acc': avg_acc}
        return results

In [32]:
resnet=resnet18(pretrained=True)

class Net(TrainNet,ValidationNet, TestNet):
  def __init__(self,batch_size=256):
    super().__init__()
    self.batch_size=batch_size
    self.conv=resnet
    self.fc=nn.Linear(1000,n_classes)
    for param in self.conv.parameters():
      param.requires_grad=False
      
  def lossfun(self, y, t):
      return F.cross_entropy(y, t)

  def configure_optimizers(self):
      return torch.optim.SGD(self.parameters(), lr=0.01)

  def forward(self, x):
      x = self.conv(x)
      x = self.fc(x)
      return x

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [33]:
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(0)

net=Net()
trainer=Trainer(gpus=1,max_epochs=10,batch_size=1024)
trainer.fit(net)

Validation sanity check:   0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

1

# Results

In [34]:
trainer.test()
trainer.callback_metrics

Testing:   0%|          | 0/3 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
TEST RESULTS
{}
----------------------------------------------------------------------------------------------------


{'epoch': 9,
 'loss': 2.359625816345215,
 'test_acc': 0.31970489025115967,
 'test_loss': 2.8550338745117188,
 'val_acc': 0.34817713499069214,
 'val_loss': 2.842740774154663}